In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow import cast,float32
from keras import backend as kb
from statistics import mean, stdev
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso

2023-04-09 09:23:54.578019: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def read_region(region):
    da = pd.read_csv("datasets/full_length_reads.csv")
    file_handle = open("datasets/"+region+".fasta","r")
    seq = []
    seqid = []
    tmp_seq = ""
    for line in file_handle:
        if (line[0] == ">"):
            if tmp_seq != "":
                seq.append(tmp_seq)
            seqid.append(line.split("\n")[0][1:])
            tmp_seq = ""
        else:
            tmp_seq+=line.split("\n")[0]
    seq.append(tmp_seq)
    file_handle.close()
    sub = pd.DataFrame([seq,seqid], index = [region,"accession"])
    sub = sub.transpose()
    da = da[["accession","copy_number"]]
    da = pd.merge(da,sub,on="accession",how="inner")
    return da

def split_test_train(da,i,multiplicand):
    X_test = da[region][i*multiplicand:(i+1)*multiplicand]
    Y_test = da['copy_number'][i*multiplicand:(i+1)*multiplicand]
    X_train = pd.concat([da[region][0:i*multiplicand],da[region][(i+1)*multiplicand:]],axis = 0)
    Y_train = pd.concat([da['copy_number'][0:i*multiplicand],
                         da['copy_number'][(i+1)*multiplicand:]],axis=0)
    return X_train,Y_train,X_test,Y_test

In [4]:
class Preprocessing():
    def __init__(self,k_size=6):
        self.k_size = k_size
        kmers = self.generate_kmers("",self.k_size)
        self.vectorizer = CountVectorizer(vocabulary = kmers)
        self.seqs = []
    
    def generate_kmers(self,current_kmer,current_depth):
        if current_depth == 1:
            return [current_kmer+"a",current_kmer+"t",current_kmer+"c",current_kmer+"g"]
        else:
            ret = self.generate_kmers(current_kmer+"a",current_depth-1)
            for nt in ['t','c','g']:
                ret += self.generate_kmers(current_kmer+nt,current_depth-1)
            return ret
    
    def generate_kmer_multiple(self,seqlist,k):
        kmer_list = []
        n = -1
        for seq in seqlist:
            kmer_list.append(self.generate_kmer_single(str(seq),k))
        return kmer_list
    
    def generate_kmer_single(self,seq,k):
        kmer = ""
        for i in range(0,len(seq)-k,1):
            kmer += seq[i:i+k]+" "
        return kmer[:-1]
    
    def CountKmers(self,seqs):
        if type(seqs) in [type([]),type(pd.core.series.Series([1]))]:
            kmer = self.generate_kmer_multiple(seqs, self.k_size)
            transformed_X = self.vectorizer.transform(kmer).toarray()
            return transformed_X
        else:
            raise ValueError("""Invalid 'seqs' format.
            Expected formats are 'list' or 'pandas.core.series.Series'.""")
            
    def ReadFASTA(self,filename,as_pd=True):
        if filename.split(".")[-1] not in ["fasta","fna","fa"]:
            raise ValueError('Invalid file format. Expected formats are ["fasta","fna","fa"].')
        file_handle = open(filename,"r")
        seqs = []
        seqid = []
        tmp_seq = ""
        for line in file_handle:
            if (line[0] == ">"):
                if tmp_seq != "":
                    seqs.append(tmp_seq)
                seqid.append(line.split("\n")[0][1:])
                tmp_seq = ""
            else:
                tmp_seq+=line.split("\n")[0]
        seqs.append(tmp_seq)
        file_handle.close()
        if as_pd:
            fasta = {}
            for i in range(len(seqs)):
                fasta[seqid[i]] = seqs[i]
            return pd.DataFrame(fasta,index=["sequence"]).transpose()["sequence"]
        else:
            return seqs, seqid

In [5]:
class CopyNumberPredictor():
    def __init__(self,region):
        self.region = region
        self.state = [59, 0.00016770313599, 0.11, 100, 489, 1, 926, 0, 645, 0, 929, 3, 582, 1, 82, 4]
        self.activation_indices={0:"relu",1:"gelu",2:"selu",3:"elu",4:"linear"}
        self.mlp = self.create_mlp()
        self.ridge = Ridge(alpha = 49)
        self.pca = PCA(n_components=100)
        self.svr = SVR(kernel='rbf',C=11,gamma='auto')
        
    def save(self,filename):
        if filename[-4:] != ".zip":
            raise ValueError('Invalid filename. Expect a zip file.')
            
        path = filename[:-4]
        prefix = path + "/" + self.region
            
        if not os.path.exists(path):
            os.makedirs(path)
        
        with open(prefix+'_pca.pkl','wb') as file:
            pickle.dump(self.pca,file)
        with open(prefix+'_ridge.pkl','wb') as file:
            pickle.dump(self.ridge,file)
        with open(prefix+'_svr.pkl','wb') as file:
            pickle.dump(self.svr,file)
        self.mlp.save(prefix+"_mlp.h5")
        
        shutil.make_archive(path, 'zip', path)
        shutil.rmtree(path)
            
    def load(self,filename):
        if filename[-4:] != ".zip":
            raise ValueError('Invalid input file. Expect a zip file.')
            
        path = filename[:-4]
        
        if not os.path.exists(path):
            os.makedirs(path)
        
        with ZipFile(filename,'r') as zObject:
            zObject.extractall(path=path)
        
        prefix = path + "/" + self.region
        with open(prefix+'_pca.pkl', 'rb') as file:
            self.pca = pickle.load(file) 
        with open(prefix+'_ridge.pkl', 'rb') as file:
            self.ridge = pickle.load(file)
        with open(prefix+'_svr.pkl', 'rb') as file:
            self.svr = pickle.load(file)
        self.mlp = load_model(prefix + "_mlp.h5",
                              custom_objects={"root_mean_squared_error": self.root_mean_squared_error})
        shutil.rmtree(path)
        
    def fit(self,X_train,Y_train,verbose=True):
        self.echo(text = "------Training Starts------", verbose = verbose)
        X_train_pca = self.pca.fit_transform(X_train)
        self.fit_mlp(X_train,Y_train)
        mlp_pred = self.mlp.predict(X_train,verbose=0)
        self.echo(text = "Model 1: MLP done.", verbose = verbose)
        
        reshaped_Y_train = Y_train.values.reshape(Y_train.shape[0])
        new_X_train = pd.concat([pd.DataFrame(X_train_pca),pd.DataFrame(mlp_pred)], axis = 1)
        
        signals = ["Model 2: SVR done."]
        for model in [self.svr]:
            model.fit(X_train_pca,reshaped_Y_train)
            pred = model.predict(X_train_pca)
            new_X_train = pd.concat([new_X_train, pd.DataFrame(pred)], axis = 1)
            self.echo(text = signals[0], verbose = verbose)
            del signals[0]
        
        self.ridge.fit(new_X_train,reshaped_Y_train)
        self.echo(text = "Meta-Model: Ridge done.", verbose = verbose)
    
    def echo(self,text,verbose):
        if verbose not in [True,False]:
            raise ValueError('verbose must be True or False')
        if verbose:
            print(text)
        
    def predict(self, X_test):
        X_test_pca = self.pca.transform(X_test)
        mlp_pred = self.mlp.predict(X_test,verbose=0)
        new_X_test = pd.concat([pd.DataFrame(X_test_pca),pd.DataFrame(mlp_pred)], axis = 1)
        
        for model in [self.svr]:
            pred = model.predict(X_test_pca)
            new_X_test = pd.concat([new_X_test, pd.DataFrame(pred)], axis = 1)
            
        final_pred = self.ridge.predict(new_X_test)
        return final_pred
    
    def create_mlp(self):
        learning_rate = self.state[1]
        model = Sequential()
        for i in range(4,len(self.state),2):
            n_neurons = self.state[i]
            activation = self.activation_indices[self.state[i+1]]
            if n_neurons != 0:
                model.add(Dense(n_neurons,activation=activation))
        model.add(Dense(1, activation="linear"))
        model.compile(loss=self.root_mean_squared_error,optimizer=Adam(learning_rate))
        return model
    
    def root_mean_squared_error(self, y_true, y_pred):
        y_true = cast(y_true,float32)
        return kb.sqrt(kb.mean(kb.square(y_pred - y_true)))
    
    def fit_mlp(self,X_train,Y_train):
        epochs = self.state[0]
        validation_split = self.state[2]
        batch_size = self.state[3]
        self.mlp.fit(X_train,Y_train,validation_split=validation_split,
                     batch_size=batch_size,epochs=epochs,verbose=0)

In [6]:
performance = {}
pp = Preprocessing()
for region in ["V1-V2","V1-V3","V3-V4","V4","V4-V5","V6-V8","V7-V9"]:
    rmse = []
    da = read_region(region)
    multiplicand = int(da.shape[0]*0.2)+1
    for i in range(0,5,1):
        X_train,Y_train,X_test,Y_test=split_test_train(da,i,multiplicand)
        model = CopyNumberPredictor()
        X_train = pp.CountKmers(seqs=X_train.tolist())
        X_test = pp.CountKmers(seqs=X_test.tolist())
        model.fit(X_train,Y_train,verbose=False)
        pred = model.predict(X_test)
        rmse.append(sqrt(mean_squared_error(Y_test,pred)))
        print("{}: {}".format(region,str(rmse[i])))
        pred_records = np.array([Y_test,pred]).transpose()
        pred_records = pd.DataFrame(pred_records)
        pred_records.columns = ["Y_test","Y_pred"]
        pred_records["group"] = i
        pred_records.to_csv("pred_records/{}_pred_records_{}.csv".format(region,str(i)),index=False)
    performance[region] = rmse

2023-04-09 09:23:57.514608: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 09:23:58.220387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3d:00.0, compute capability: 7.5


V1-V2: 0.7268926161131259
V1-V2: 0.8002456196603392
V1-V2: 0.7438915385524352
V1-V2: 0.7431233281209279
V1-V2: 0.74003240707514
V1-V3: 0.7062944705512825
V1-V3: 0.7711177750120262
V1-V3: 0.6939612654542386
V1-V3: 0.7250750040210533
V1-V3: 0.739706259557142
V3-V4: 0.7011701423123204
V3-V4: 0.7921285526295864
V3-V4: 0.6738289324552839
V3-V4: 0.7376702803588326
V3-V4: 0.7442644842110111
V4: 0.7423095215886508
V4: 0.8439847256265849
V4: 0.7266671597553932
V4: 0.7811231330050725
V4: 0.7740247582464207
V4-V5: 0.7204780655739282
V4-V5: 0.7953225419731003
V4-V5: 0.7021554596980343
V4-V5: 0.7629517733024095
V4-V5: 0.7477324864504962
V6-V8: 0.7191177801498575
V6-V8: 0.7886024538913772
V6-V8: 0.6868535511593724
V6-V8: 0.7342963687742476
V6-V8: 0.7367671121462038
V7-V9: 0.7085002894956247
V7-V9: 0.783985268294651
V7-V9: 0.6799405191631711
V7-V9: 0.7630053269285263
V7-V9: 0.7518238422118043


In [7]:
pd.DataFrame(performance).mean(axis = 0)

V1-V2    0.750837
V1-V3    0.727231
V3-V4    0.729812
V4       0.773622
V4-V5    0.745728
V6-V8    0.733127
V7-V9    0.737451
dtype: float64

In [8]:
pd.DataFrame(performance).to_csv("performance/smac_universal_subregions.csv",index=False)